In [60]:
import pandas as pd
import os
DATA_FOLDER = {
    "Phoenix": "src/data/valley_metro",
    "Boston": "src/data/mbta",
}

In [61]:
city= "Boston"  # Example city, can be changed to "Boston"

In [62]:
gtfs_columns = {
    "trips.txt": ["route_id", "trip_id", "trip_headsign", "shape_id"],
    "routes.txt": ["route_id", "route_short_name", "route_color"],
    "shapes.txt": None  # Load all columns
}

# Load the GTFS data
data_path = DATA_FOLDER[city]
trips_df = pd.read_csv(os.path.join(data_path, "trips.txt"), usecols=gtfs_columns["trips.txt"])
routes_df = pd.read_csv(os.path.join(data_path, "routes.txt"), usecols=gtfs_columns["routes.txt"])
shapes_df = pd.read_csv(os.path.join(data_path, "shapes.txt"))

/var/folders/nr/1tw88k6s4xg8ng602yz6f4h00000gn/T/ipykernel_64928/791343193.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  shapes_df = pd.read_csv(os.path.join(data_path, "shapes.txt"))


In [63]:
trips_df.dtypes

route_id         object
trip_id          object
trip_headsign    object
shape_id         object
dtype: object

In [64]:
routes_df["route_id"] = routes_df["route_id"].astype(str)
trips_df["route_id"] = trips_df["route_id"].astype(str)
trips_df["trip_id"] = trips_df["trip_id"].astype(str)
trips_df = pd.merge(trips_df, routes_df, on='route_id')

In [65]:
shape_paths_df = (
    shapes_df.sort_values(by=["shape_id", "shape_pt_sequence"])
      .groupby("shape_id")
      .apply(lambda g: [[lon, lat] for lon, lat in zip(g["shape_pt_lon"], g["shape_pt_lat"])])
      .reset_index(name="route_path")
)

/var/folders/nr/1tw88k6s4xg8ng602yz6f4h00000gn/T/ipykernel_64928/2675517433.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: [[lon, lat] for lon, lat in zip(g["shape_pt_lon"], g["shape_pt_lat"])])


In [66]:
trips_df = pd.merge(trips_df, shape_paths_df, on='shape_id', how='left')
trips_df.columns

Index(['route_id', 'trip_id', 'trip_headsign', 'shape_id', 'route_short_name',
       'route_color', 'route_path'],
      dtype='object')

In [67]:
trips_df.isna().sum()

route_id                0
trip_id                 0
trip_headsign           0
shape_id                0
route_short_name     9297
route_color             0
route_path          20333
dtype: int64

In [68]:
def hex_to_rgb(h):
    h = h.lstrip("#")
    return tuple(int(h[i : i + 2], 16) for i in (0, 2, 4))

In [69]:
trips_df["route_color"] = trips_df["route_color"].apply(hex_to_rgb)
trips_df

,route_id,trip_id,trip_headsign,shape_id,route_short_name,route_color,route_path
0,1,69696876,Harvard,010150,1,"(255, 199, 44)",NaN
1,1,69696879,Harvard,010150,1,"(255, 199, 44)",NaN
2,1,69696881,Harvard,010150,1,"(255, 199, 44)",NaN
3,1,69696889,Harvard,010150,1,"(255, 199, 44)",NaN
4,1,69696893,Harvard,010150,1,"(255, 199, 44)",NaN
...,...,...,...,...,...,...,...
45323,Shuttle-StateSullivan,StateSullivan-1-sample,Sullivan (Shuttle),StateToSullivan-S,Orange Line Shuttle,"(255, 199, 44)","[[-71.0568, 42.359], [-71.0568, 42.35905], [-7..."
45324,Shuttle-SuffolkDownsWonderland,SuffolkDownsWonderland-0-sample,Suffolk Downs (Shuttle),WonderlandToSuffolkDowns-S,Blue Line Shuttle,"(255, 199, 44)","[[-70.99213, 42.41359], [-70.99219, 42.41367],..."
45325,Shuttle-SuffolkDownsWonderland,SuffolkDownsWonderland-1-sample,Wonderland (Shuttle),SuffolkDownsToWonderland-S,Blue Line Shuttle,"(255, 199, 44)","[[-70.99688, 42.39017], [-70.99657, 42.39042],..."
45326,Shuttle-TFGreenWickfordJunction,TFGreenWickfordJunction-0-sample,Wickford Junction (Shuttle),TFGreenToWickfordJunction-S,Providence Line Shuttle,"(255, 199, 44)","[[-71.44345, 41.72757], [-71.44334, 41.72785],..."
